# LINEAR REGRESSION 

## CHAPTER #1 - HANDLING DATA 

## Importing dependencies 

In [ ]:
# Importing necessary packages 
%pip install pandas 
%pip install numpy
%pip install matplotlib
%pip install scikitlearn

In [ ]:
import numpy as np 
import matplotlib as mpl
import pandas as pd
from matplotlib import pyplot as plt

## Loading data 
The following datasets provide metrics partitioned by market capitalization, price, volatility, and turnover. The stock market activity metrics are partitioned by decile and the ETP metrics by quartile. 

I want to look into how cancellation rate is affected by stock volatility.
I will loook at decile 1 (the lowest market capitilsation of 10 businesses) and the Market cap decile column as well as its Volatility to see how cancellation rate changes with volatility. 
I would hypothesise that the greater the volaitilty the greater the rate of cancellation. 

In our data schema the following are defined:  
Market Cap  Decile1- what is the decile_cancel_to_trade (number of cancelled trades/ number of successful trades) for that capitalisation at that date.    
Volatility- the amount statsitical variation within in stock decile (e.g decile 1) as that date. 

## Can we predict the the cancelllation rate of a stock based on its volatility?
We will use linaer regression to find out.

In [ ]:
decile_path= "/Users/admin/Desktop/Data Science Career /Python/Python Projects/Linear regression from scratch /decile_quartile_2025_q1/decile_cancel_to_trade_stock.csv"
#Saving path name as variable for read csv argument 

In [ ]:
decile_to_cancel_raw=pd.read_csv(decile_path)

In [ ]:
decile_to_cancel_raw.head()
decile_to_cancel_raw.tail()
#A little insight into what our data looks like 

## Pre-processing and cleaning 


-For the purposes of this model we will limit our dataset to decile 1.   
This is to isloate only volatility and cancellation features for one set of independent and dependent variables. 

In [ ]:
decile_1 = decile_to_cancel_raw[["Market Cap Decile1","Volatility Decile1"]] 
#extracting features of market cap decile 1
#independent variable = volatility decile 1 
#dependent varaible = market cap decile1

In [ ]:
decile_1.head()

In [ ]:
help(decile_1.rename) #help on how to rename columns 

In [ ]:
decile_1 = decile_1.round(2)
decile_1.head()

In [ ]:
decile_1 = decile_1.rename(columns={'Market Cap Decile1':'Cancellation rate','Volatility Decile1':'Volatility'})
#renaming columns since we know we are in decile 1 of the canccel to trade file

## Linear Regression Assumptions
In linaer regression we assume a few key aspects of our data:   
1) Linearity of the data points. 
2) Homoscedacity   
3) Normality of erros   
 
To test the first asusumptions I will make a basic plot of the two features.

### Linearity
Plot our indepdent variable vs depedent variable as a scatterplot 

In [ ]:
x_values = decile_1[['Cancellation rate']]
y_values = decile_1[['Volatility']]

In [ ]:
plot_linear = plt.scatter(y_values, x_values)

From this plot we can see a few key details, namely: 
1) Our data has a few outliers.  
2) Our data does follow a linear relationship with most values condensed around the centre point of the volatility scale.  
3) The linearity is present but does not have a strong gradient meaning the volaitly in decile 1 does not have much predictive power in rehgards to the cencellation rate. I will explore  different features for this model. 

In [ ]:
decile_9 = decile_to_cancel_raw[["Market Cap Decile9","Volatility Decile9"]] 

In [ ]:
y_values_2 = decile_9[['Market Cap Decile9']]
x_values_2 = decile_9[['Volatility Decile9']]

In [ ]:
plot_linear = plt.scatter(x_values_2, y_values_2)

From this plot we can see that:
1) There is a string positive linear relationship, therefore volatililty does have predictive power for cancellation rates. 
2) There is heteroscedacity in the raw data, therefore we may need to apply some kind of transformation to the data but I will make the regression model and then check for homoscedacity in the residuals. 
3) Interetsing though since this high heteroscedcaity indicates that as stock get more volatile purchasing decisions become more extreme. 

In [ ]:
decile_9 = decile_9.rename(columns={'Market Cap Decile9':'Cancellation rate','Volatility Decile9':'Volatility'})
#renaming columns since we know we are in decile 9 of the cancel_to_trade file

In [ ]:
plot_box = plt.boxplot(x_values_2)

In [ ]:
plot_box = plt.boxplot(y_values_2)

From these plots we can see the presence of:
1) A significant number of outliers outside the maximum range of the dataset.
2) We will use the interquartile range method to impute these values.

## OUTLIER REMOVAL 
I will use the method of removing values above using upper and lower bounds based on quartiles. 

In [ ]:
#Step 1- compute Q1 and Q3
#Cancellation rates 
Q1C = decile_9['Cancellation rate'].quantile(0.25)
Q3C = decile_9['Cancellation rate'].quantile(0.75)
print(Q1C)
print(Q3C)




In [ ]:
#Volatiltiy 
Q1V = decile_9['Volatility'].quantile(0.25)
Q3V = decile_9['Volatility'].quantile(0.75)
print(Q1V)
print(Q3V)

In [ ]:
#Step 2 - Compute IQR
#Cancellation rate 
IQRC = Q3C - Q1C
print(IQR)

In [ ]:
#Volatility 
IQRV= Q3V -Q1V
print(IQRV)

In [ ]:
#Step 3 - Find the upper bound and lower bound
#Cancellation rate
upper_b_Canc = Q3C + 1.5*IQRC
print(upper_b_Canc)

In [ ]:
#Volatility 
upper_b_Vol = Q3V + 1.5*IQRV
print(upper_b_Vol)

In [ ]:
#Number of outliers - https://www.analyticsvidhya.com/blog/2022/09/dealing-with-outliers-using-the-iqr-method/
#Cancellation rate
decile_9[decile_9['Cancellation rate'] > upper_b_Canc].count()
print((70/3329)*100) #percentage of values to impute. Is not excessive ≈ 2%

In [ ]:
#Volatility
decile_9[decile_9['Volatility'] > upper_b_Vol].count()
print((69/3329)*100) #percentage of values to impute. Is not excessive ≈ 2%

In [ ]:
#Capping outlier values- Since these values represent only upper tail outlers, truncating them would remove information from the data and make our model have 
#lower predictive power at the high end. Therefore I wil use Winsorization to cap them -https://www.datacamp.com/tutorial/winsorized-mean
#Cancellation rate 

decile_9['Cancellation rate'] = decile_9['Cancellation rate'].clip (upper = upper_b_Canc)
decile_9[decile_9['Cancellation rate'] > upper_b_Canc].count() #success 


In [ ]:
decile_9['Volatility'] = decile_9['Volatility'].clip (upper = upper_b_Vol)
decile_9[decile_9['Volatility'] > upper_b_Vol].count() #success 

In [ ]:
# New distribution plots
y = decile_9[['Cancellation rate']]
x = decile_9[['Volatility']]

In [ ]:
plt.boxplot(x)

In [ ]:
plt.boxplot(x)

In [ ]:
plt.hist(x)

In [ ]:
plt.hist(y)

Now that we have the pre-processed data with linearity and outliers handled we can create our linear regression model

## CHAPTER #2 - LINEAR REGRESSION MODEL 

 ### CLASS CREATION  
 This class will contain all of the model logic.   
 The class contains the Linear regression model itself. 
 Each object is a learned model i.e a line for each set of training data   

 The attributes contain the:  
 - weight (coefficeint of the independent variable)  
 - bias (y-intercept) of the line 

 The methods contain all the verbs of the model (functions that change the weight and bias), namely:  
 - how the model learn weight and bias. 
 - how the model fits the line to the data   
 - how the line is evaluated e.g R^2 and RMSE. 
 - importsnt data about the model like residuals 


In [ ]:
class  LinearRegression(object):
    def __init__(self, weight=0, bias=0): #initialises the attributes of the class
        self.weight = weight              #stores weight 
        self.bias = bias                  #stores bias 
        self.x = []                       #stores our predictor variables (x) as lists  
        self.y = []                       #stores our predicted variable (y) as lists 
 
    def vectorise(self, x,y):             #defining method to store the data points to be modelled
        self.x = x                        #storing the values of x independent variable within the class 
        self.y = y                        #storing the values of y dependent variable within the class 

    def predict_y (self):                 #calcualting the predicted y[i] for our optimisation later 
        y_predict =[]                     #creating an empty list to store all predicted y values 
        n = len(self.y)

        for i in range(n):                #looping over the number of values we have in the dataset 
            y_predict.append(self.weight*self.x[i] + self.bias)     #calculating predicted y values with line equation and adding predicted values to our list 
        return y_predict                  

#Creating method to get weight
    def partial_w(self):                  #partial derivative in regard to weight 
        y_predict = self.predict_y()
        gradient = 0 
        n=len(self.y)

        for i in range(n):
            gradient += self.x[i]*(y_predict[i] - self.y[i])         #partial derivative equation to calculate total partial derivative of weight in regards to error function
        return (-2/n)*gradient

#Creating method to get bias 
    def partial_b(self):
        y_predict = self.predict_y()
        gradient = 0
        n=len(self.y)

        for i in range(n):
            gradient += (y_predict[i]- self.y[i])                     #partial derivative equation to calculate total partial derivative of bias in regards to error function
        return (-2/n)*gradient

#Numerical optimisation via gradient descent 
    def optimise(self): 
        learn_rate = 0.005                 #size of steps we make "downhill" to minimise total error in regards to the weight and bias 

        for i in range(10000):           #number of "epochs" we take steps in to minimise aggregate error 
            self.weight = self.weight + learn_rate * self.partial_w() #optimised weight 
            self.bias = self.bias + learn_rate * self.partial_b()     #optimised bias 
            if i % 10 == 0:                #prints out the weight and bias every 10 epochs 
                print(self.weight, self.bias)
    
#Residual calculations 
    def residuals(self):
        residuals = []
        n=len(self.x)

        for i in range(n):
            residuals.append(self.y[i] - (self.weight * self.x[i] + self.bias))
        return residuals

#Evaluation metrics
#Mean Square Error  

    def mse(self):
        mse = 0
        mse_list = []
        n=len(self.y)
        total_error = []

        for i in range(n):
            total_error.append((self.y[i] - (self.weight * self.x[i] + self.bias))**2)
            mse_list.append((1/n)*total_error[i])
            mse += mse_list[i]
        return mse 
#R^2
    def rsquared(self):
        n=len(self.y)
        avg_y = 0

        for i in range(n):
            avg_y += ((1/n)*self.y[i])
    



        
        

        
        

            

            











 




EXAMPLE WITH SIMPLE LISTS

In [ ]:
x = [1,2,3,4,5]
y = [6,7,8,9,10]



In [ ]:
model = LinearRegression()

In [ ]:
model.vectorise(x,y)

In [ ]:
model.optimise()

In [ ]:
model.predict_y()

In [ ]:
model.residuals()

In [ ]:
model.mse()